# Lab4-Assignment about Named Entity Recognition and Classification

This notebook describes the assignment of Lab 4 of the text mining course. We assume you have succesfully completed Lab1, Lab2 and Lab3 as welll. Especially Lab2 is important for completing this assignment.

**Learning goals**
* going from linguistic input format to representing it in a feature space
* working with pretrained word embeddings
* train a supervised classifier (SVM)
* evaluate a supervised classifier (SVM)
* learn how to interpret the system output and the evaluation results
* be able to propose future improvements based on the observed results


## Credits
This notebook was originally created by [Marten Postma](https://martenpostma.github.io) and [Filip Ilievski](http://ilievski.nl) and adapted by Piek vossen

## [Points: 18] Exercise 1 (NERC): Training and evaluating an SVM using CoNLL-2003

**[4 point] a) Load the CoNLL-2003 training data using the *ConllCorpusReader* and create for both *train.txt* and *test.txt*:**

    [2 points]  -a list of dictionaries representing the features for each training instances, e..g,
    ```
    [
    {'words': 'EU', 'pos': 'NNP'}, 
    {'words': 'rejects', 'pos': 'VBZ'},
    ...
    ]
    ```

    [2 points] -the NERC labels associated with each training instance, e.g.,
    dictionaries, e.g.,
    ```
    [
    'B-ORG', 
    'O',
    ....
    ]
    ```

In [1]:
from nltk.corpus.reader import ConllCorpusReader
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader('CONLL2003', 'train.txt', ['words', 'pos', 'ignore', 'chunk'])
training_features = []
training_gold_labels = []

for token, pos, ne_label in train.iob_words():
    a_dict = {
        'words': token,
        'pos': pos,
        'labels': ne_label
       # add features
    }
    training_features.append({'words': a_dict['words'], 'pos': a_dict['pos']})
    training_gold_labels.append(a_dict['labels'])
print(training_features[:10])
training_gold_labels[:10]
print(len(training_features))
   

[{'words': 'EU', 'pos': 'NNP'}, {'words': 'rejects', 'pos': 'VBZ'}, {'words': 'German', 'pos': 'JJ'}, {'words': 'call', 'pos': 'NN'}, {'words': 'to', 'pos': 'TO'}, {'words': 'boycott', 'pos': 'VB'}, {'words': 'British', 'pos': 'JJ'}, {'words': 'lamb', 'pos': 'NN'}, {'words': '.', 'pos': '.'}, {'words': 'Peter', 'pos': 'NNP'}]
203621


In [2]:
### Adapt the path to point to the CONLL2003 folder on your local machine
test = ConllCorpusReader('CONLL2003', 'test.txt', ['words', 'pos', 'ignore', 'chunk'])

test_features = []
test_gold_labels = []
for token, pos, ne_label in test.iob_words():
    a_dict = {
        'words': token,
        'pos': pos,
        'labels': ne_label
       # add features
    }
    test_features.append({'words': a_dict['words'], 'pos': a_dict['pos']})
    test_gold_labels.append(a_dict['labels'])
print(test_features[:10])
print(test_gold_labels[:10])


[{'words': 'SOCCER', 'pos': 'NN'}, {'words': '-', 'pos': ':'}, {'words': 'JAPAN', 'pos': 'NNP'}, {'words': 'GET', 'pos': 'VB'}, {'words': 'LUCKY', 'pos': 'NNP'}, {'words': 'WIN', 'pos': 'NNP'}, {'words': ',', 'pos': ','}, {'words': 'CHINA', 'pos': 'NNP'}, {'words': 'IN', 'pos': 'IN'}, {'words': 'SURPRISE', 'pos': 'DT'}]
['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O']


**[2 points] b) provide descriptive statistics about the training and test data:**
* How many instances are in train and test?
* Provide a frequency distribution of the NERC labels, i.e., how many times does each NERC label occur?
* Discuss to what extent the training and test data is balanced (equal amount of instances for each NERC label) and to what extent the training and test data differ?

Tip: you can use the following `Counter` functionality to generate frequency list of a list:

In [3]:
from collections import Counter 
print("Training Set:\n")
print("Intances - %s\n" %(len(training_gold_labels)))
print("frequency of labels:")
for label, count in Counter(training_gold_labels).items():
    print('%s - %s'%(label,round((count/len(training_gold_labels))*100,2)))
print("\n\nTest Set:\n")
print("Instances - %s\n" %(len(test_gold_labels)))
print("frequency of labels:")
for label, count in Counter(test_gold_labels).items():
    print('%s - %s'%(label,round((count/len(test_gold_labels))*100,2)))

Training Set:

Intances - 203621

frequency of labels:
B-ORG - 3.1
O - 83.28
B-MISC - 1.69
B-PER - 3.24
I-PER - 2.22
B-LOC - 3.51
I-ORG - 1.82
I-MISC - 0.57
I-LOC - 0.57


Test Set:

Instances - 46435

frequency of labels:
O - 82.53
B-LOC - 3.59
B-PER - 3.48
I-PER - 2.49
I-LOC - 0.55
B-MISC - 1.51
I-MISC - 0.47
B-ORG - 3.58
I-ORG - 1.8


## Answer
The training set has 203621 instances whilst the test set has 46435 which is around a 80-20 split. 

in terms of how often a label occurs in its respective set, both the training and test set seemed balanced overall. The label 'O' occurs 83% percent of the time in the training set and 82.5% in the test set. these close comparisons happens for all labels. 

The only difference between the 2 sets is the overall amount of instances as mentioned before.

**[2 points] c) Concatenate the train and test features (the list of dictionaries) into one list. Load it using the *DictVectorizer*. Afterwards, split it back to training and test.**

Tip: You’ve concatenated train and test into one list and then you’ve applied the DictVectorizer.
The order of the rows is maintained. You can hence use an index (number of training instances) to split the_array back into train and test. Do NOT use: `
from sklearn.model_selection import train_test_split` here.


In [4]:
!pip install sklearn

In [5]:
from sklearn.feature_extraction import DictVectorizer

In [6]:
vec = DictVectorizer()
cutoff = len(training_features)
the_array = training_features
the_array.extend(test_features)
the_array = vec.fit_transform(the_array)
training = the_array[:cutoff]
testing = the_array[cutoff:]
# your code here

**[4 points] d) Train the SVM using the train features and labels and evaluate on the test data. Provide a classification report (sklearn.metrics.classification_report).**
The train (*lin_clf.fit*) might take a while. On my computer, it took 1min 53s, which is acceptable. Training models normally takes much longer. If it takes more than 5 minutes, you can use a subset for training. Describe the results:
* Which NERC labels does the classifier perform well on? Why do you think this is the case?
* Which NERC labels does the classifier perform poorly on? Why do you think this is the case?

In [7]:
from sklearn import svm

In [8]:
lin_clf = svm.LinearSVC()

In [9]:
##### [ YOUR CODE SHOULD GO HERE ]
from sklearn.metrics import classification_report
lin_clf.fit(training, training_gold_labels)
predict = lin_clf.predict(testing)
print(classification_report(test_gold_labels, predict))

              precision    recall  f1-score   support

       B-LOC       0.81      0.78      0.79      1668
      B-MISC       0.78      0.66      0.72       702
       B-ORG       0.79      0.52      0.63      1661
       B-PER       0.86      0.44      0.58      1617
       I-LOC       0.62      0.53      0.57       257
      I-MISC       0.57      0.59      0.58       216
       I-ORG       0.70      0.47      0.56       835
       I-PER       0.33      0.87      0.48      1156
           O       0.98      0.98      0.98     38323

    accuracy                           0.92     46435
   macro avg       0.72      0.65      0.65     46435
weighted avg       0.94      0.92      0.92     46435



**[6 points] e) Train a model that uses the embeddings of these words as inputs. Test again on the same data as in 2d. Generate a classification report and compare the results with the classifier you built in 2d.**

In [13]:
%%bash 
pip install -U gensim

In [11]:
%%bash
python --version

Python 3.9.13


In [12]:
# your code here
import gensim
word_embedding_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

ModuleNotFoundError: No module named 'gensim'

In [ ]:
#training_words = []
#for token, pos, ne_label in train.iob_words():
    #a_dict = {
        #'words': token
    #}
    #training_words.append(a_dict['words'])

#test_words = []
#for token, pos, ne_label in test.iob_words():
    #a_dict = {
        #'words': token
       # add features
    #}
    #test_words.append(a_dict['words'])
    
#print(test_words)

In [ ]:
#both_input=[]
#for token in (training_words + test_words): #the next word from the tokenized text
    # we check if our word 
    # is inside the model vocabulary (loaded with the Google word2vec embeddings)
    #if token in word_embedding_model:
        # in this case the word was found and vector is assigned with its embedding vector as the value
        #vector=word_embedding_model[token]
    #else: 
        # if the word does not exist in the embeddings vocabulary, 
        # we create a vector with all zeros.
        # The Google word2vec model has 300 dimensions so we creat a vector with 300 zeros
        #vector=[0]*300
        #print('This word is not in the word2vec vocabulary:', token)
    #both_input.append(vector)


## [Points: 10] Exercise 2 (NERC): feature inspection using the [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)
**[6 points] a. Perform the same steps as in the previous exercise. Make sure you end up for both the training part (*df_train*) and the test part (*df_test*) with:**
* the features representation using **DictVectorizer**
* the NERC labels in a list

Please note that this is the same setup as in the previous exercise:
* load both train and test using:
    * list of dictionaries for features
    * list of NERC labels
* combine train and test features in a list and represent them using one hot encoding
* train using the training features and NERC labels

In [14]:
import pandas

In [16]:
##### Adapt the path to point to your local copy of NERC_datasets
path = 'ner_dataset.csv'
kaggle_dataset = pandas.read_csv(path, on_bad_lines='skip')

/var/folders/44/42krrmz573z3bz3qw2zt33lw0000gn/T/ipykernel_51148/3935160730.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  kaggle_dataset = pandas.read_csv(path, error_bad_lines=False)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 0: invalid start byte

In [ ]:
len(kaggle_dataset)

In [ ]:
df_train = kaggle_dataset[:100000]
df_test = kaggle_dataset[100000:120000]
print(len(df_train), len(df_test))

**[4 points] b. Train and evaluate the model and provide the classification report:**
* use the SVM to predict NERC labels on the test data
* evaluate the performance of the SVM on the test data

Analyze the performance per NERC label.

## End of this notebook